In [1]:
import json
import pandas as pd
import numpy as np
import os

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

copy complete until step 6; once step 6 was completed there were 7037 rows, which is 4 more than the 7033 listed on 8.3.7 of the module.  Not sure where I went wrong. 

Before step 6, I had 7080 results, matching with module at 8.3.3

In [17]:
# 1. Add the clean movie function that takes in the argument, "movie".
# Added from lessons8.3.6 clean movies combine the earlier function with the new
#merge columns function

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [18]:
#delete later: create file path for testing

# 6 Create the path to your file directory and variables for the three files. 
file_dir = os.path.join("../Movies-ETL")
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [19]:
#step 2 testing block

def ETL():
    
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_file = pd.read_csv('movies_metadata.csv', low_memory = False)
    ratings_file = pd.read_csv('ratings.csv', low_memory = False)

    # 3. Open the read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia_movies.json', mode='r') as file:
        wiki_movies = json.load(file)
    
     # 3. Write a list comprehension to filter out TV shows.
    wiki_file = [movie for movie in wiki_movies
              if('Director' in movie or 'Directed by' in movie)
              and 'imdb_link' in movie
              and "no. of episodes" not in movie]
     # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_file]
    
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    clean_wiki_df = pd.DataFrame(clean_movies)
    sorted(clean_wiki_df.columns.tolist())
    
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        clean_wiki_df['imdb_id'] = clean_wiki_df['imdb_link'].str.extract(r'(tt\d{7})')
        clean_wiki_df.drop_duplicates(subset='imdb_id', inplace=True)
        
    except(IndexError):
        print('Index Error .... skipping')   
        
        
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    


    
    return clean_wiki_df, kaggle_file, ratings_file

In [20]:
#function tested to step 5, len 7080 and returns a .head & .tail; both CSV are type = DF
# works up to step 6 len(clean_wiki_df = 7037 which is 4 more than module 8.3.7)
clean_wiki_df, kaggle_file, ratings_file = ETL()

In [21]:
len(clean_wiki_df)

7037

In [16]:
#check null columns
[[column,clean_wiki_df[column].isnull().sum()] for column in clean_wiki_df.columns]

[['url', 0],
 ['year', 0],
 ['imdb_link', 0],
 ['title', 1],
 ['Based on', 4855],
 ['Starring', 185],
 ['Narrated by', 6755],
 ['Cinematography', 693],
 ['Release date', 32],
 ['Running time', 139],
 ['Country', 236],
 ['Language', 248],
 ['Budget', 2298],
 ['Box office', 1552],
 ['Director', 0],
 ['Distributor', 360],
 ['Editor(s)', 548],
 ['Composer(s)', 519],
 ['Producer(s)', 203],
 ['Production company(s)', 1678],
 ['Writer(s)', 201],
 ['Genre', 6923],
 ['Original language(s)', 6875],
 ['Original network', 6908],
 ['Executive producer(s)', 6937],
 ['Production location(s)', 6987],
 ['Picture format', 6969],
 ['Audio format', 6973],
 ['Voices of', 7035],
 ['Followed by', 7028],
 ['Created by', 7026],
 ['Opening theme', 7036],
 ['No. of episodes', 7033],
 ['alt_titles', 7015],
 ['Preceded by', 7027],
 ['Suggested by', 7036],
 ['Recorded', 7035],
 ['Venue', 7036],
 ['Label', 7035],
 ['Animation by', 7035],
 ['Color process', 7036],
 ['Camera setup', 7036],
 ['McCune–Reischauer', 7035]

In [15]:
clean_wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7037 entries, 0 to 7079
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   url                     7037 non-null   object
 1   year                    7037 non-null   int64 
 2   imdb_link               7037 non-null   object
 3   title                   7036 non-null   object
 4   Based on                2182 non-null   object
 5   Starring                6852 non-null   object
 6   Narrated by             282 non-null    object
 7   Cinematography          6344 non-null   object
 8   Release date            7005 non-null   object
 9   Running time            6898 non-null   object
 10  Country                 6801 non-null   object
 11  Language                6789 non-null   object
 12  Budget                  4739 non-null   object
 13  Box office              5485 non-null   object
 14  Director                7037 non-null   object
 15  Dist

In [6]:
len(clean_wiki_df)

7080

In [9]:
clean_wiki_one = clean_wiki_df.copy()

In [10]:
len(clean_wiki_one)

7080

In [11]:
clean_wiki_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7080 entries, 0 to 7079
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   url                     7080 non-null   object
 1   year                    7080 non-null   int64 
 2   imdb_link               7080 non-null   object
 3   title                   7079 non-null   object
 4   Based on                2191 non-null   object
 5   Starring                6895 non-null   object
 6   Narrated by             283 non-null    object
 7   Cinematography          6380 non-null   object
 8   Release date            7048 non-null   object
 9   Running time            6940 non-null   object
 10  Country                 6841 non-null   object
 11  Language                6832 non-null   object
 12  Budget                  4765 non-null   object
 13  Box office              5517 non-null   object
 14  Director                7080 non-null   object
 15  Dist

pandas.core.frame.DataFrame

In [31]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)
## from Del 1
def ETL():
    
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    #kaggle_file = pd.read_csv('movies_metadata.csv', low_memory = False)
    #ratings_file = pd.read_csv('ratings.csv', low_memory = False)

    # 3. Open the read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia_movies.json', mode='r') as file:
        wiki_movies = json.load(file)
    
     # 3. Write a list comprehension to filter out TV shows.
    wiki_file = [movie for movie in wiki_movies
              if('Director' in movie or 'Directed by' in movie)
              and 'imdb_link' in movie
              and "no. of episodes" not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_file]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    #clean_wiki_df = pd.DataFrame(clean_movies)
   # sorted(wiki_movies_df.columns.tolist())
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
   # try:
        
  #  except 

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    

    # 8. Create a variable that will hold the non-null values from the “Box office” column.

    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
   
   # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    

    # 12. Add the parse_dollars function.
    
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.

    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_file
                 #clean_wiki_df #kaggle_file, ratings_file

In [4]:
# 17. Create the path to your file directory and variables for the three files.
# 6 Create the path to your file directory and variables for the three files. 
file_dir = os.path.join("../Movies-ETL")
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 18. Set the three variables equal to the function created in D1.
##changed func name to match with mine
wiki_file, kaggle_file, ratings_file = ETL()

In [6]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [7]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

In [8]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()